![Digital Futures](https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true)

# Setting up the Project Environment

Ultimately, the Python code written here will be extracted to scripts for execution in an automated pipeline.  To facilitate this, there is a need to set up a project environment that will allow for the execution of the code in a controlled and reproducible environment.

In the initial stages of the activities, the packages needed are `requests` and `pytest`.  The `requests` package is used to make HTTP requests to the API, while `pytest` is used for testing the code we also need *BeautifulSoup* (package name `beautifulsoup4`.  In later activities, you may need to install additional packages.  To do this, add the packages to the `pip install` command below and re-run the cell.

> **Remember:** The goal is to create a set of code cells that can be extracted to separate scripts for execution in an automated pipeline.  Therefore, the code should be kept in 3 distinct cells:
> 
> - **Shell Commands**:  Used to set up the project environment
> 
> - **Python Tests**: Used to test the Python production scripts both now and as part of the automated pipeline
> 
> - **Python Production Code**: The Python code that will be extracted to a script to execute during the pipeline

---

# Environment Setup Scripts

If you are running this notebook after cloning and have not set up your environment to run shell commands, you will need to run the following commands in your terminal to set up the environment.

> **NOTE:**  These commands need to be executed in the terminal.  
>
> Open a terminal at the root of your project before executing these commands
> 
> Until your environment is set up, Jupyter Notebooks will not be able to run **shell** scripts.

```sh
# Create a virtual environment (add the command below)
python3 -m venv .venv # Note: This command could also be python -m venv .venv # python3 and python are a symlink to the python version installed on your system

# Activate the virtual environment 
source .venv/bin/activate

# Install required package to execute shell commands from Jupyter Notebook
pip install ipykernel               ## OR 
pip install -r requirements.txt     ## IF there is already a requirements.txt file CONTAINING ipykenrnel in the project
```


In [43]:
# Install the necessary packages
!pip install requests pytest beautifulsoup4

# Create a requirements.txt file
!pip freeze > requirements.txt

> **Note:** 
> The `!` at the beginning of the lines is a special character in Jupyter Notebooks that allows you to run shell commands from the notebook.  
> These will need to be removed from any commands that are to be exported to a `.sh` shell script file for the pipeline.

---

# Python Tests

Develop any tests for functions in separate cells below.  The first has been provided for you as an example, add others as necessary.

### Test `request_to_scrape`

In [44]:
# Test request_to_scrape
from unittest.mock import *
import unittest


def test_request_to_scrape():
    # arrange
    url = "http://books.toscrape.com/"
    with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 200

    # act 
        request_to_scrape(url)

    # assert
    
        mock_get.assert_called_once_with(url) 
        

def test_request_to_scrape_return():
    # arrange
    url = "http://www.example.com/"
    text = "<html><body><h1>Hello, World!</h1></body></html>"
    with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 200
        mock_get.return_value.text = text

    # act 
        response = request_to_scrape(url)

    # assert
        assert response == text
        # assert response.status_code == mock_get.return_value.status_code
        
def test_request_to_scrape_fail():
    # arrange
    url = "http://books.toscrape.com/"
    with patch('requests.get') as mock_get:
        mock_get.return_value.status_code = 404

    # act 
        result = request_to_scrape(url)

    # assert
        assert result == "Error: Request failed with status code 404"
       
def test_request_to_scrape_get_fail():
    # arrange
    url = "http://books.toscrape.com/"
    with patch('requests.get') as mock_get:
        mock_get.side_effect = requests.exceptions.RequestException("Failed to connect")

    # act 
        response = request_to_scrape(url)

    # assert
    
        assert response == "Error occurred: Failed to connect"

### Run the tests

Run the cell containing the `ipytest.run()` command to execute the tests.  The tests should all fail until you have written the production code.

Don't forget to run the installation and initialisation cell too on the first time you run the tests!


---

# Python Production Code


Develop any functions for use as production code in separate cells below. The first has been provided as an example under the Production Constants, add others as necessary.

### PRODUCTION CONSTANTS

In [45]:
# PRODUCTION CONSTANTS

# Constants for status messages
STATUS_SUCCESS = "success"
STATUS_ERROR = "error"
ERROR_NOT_HTML = "The response is not HTML"
ERROR_REQUEST_FAILED = "Request failed for URL"
ERROR_UNEXPECTED = "Unexpected error for URL"

# HTML Parser
HTML_PARSER = "html.parser"

### `request_to_scrape` Production Code

In [46]:
# request_to_scrape Production Code
import requests


def request_to_scrape(url):
    
    """
    Sends an HTTP GET request to the specified URL and returns the response content.

    This function handles any potential exceptions that occur during the request,
    including network issues or invalid responses. If the request is successful,
    it returns the raw HTML content of the response. If the request fails or an
    exception occurs, an appropriate error message is returned.

    Args:
        url (str): The URL to which the GET request will be sent.

    Returns:
        str: The content of the response (HTML text) if the request is successful,
             or an error message if the request fails or an exception is raised.
    
    Example:
        >>> request_to_scrape("http://example.com")
        "<html>...</html>"
        
        >>> request_to_scrape("http://invalid-url")
        "Error occurred: ..."
    """
    
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        if response.status_code == 200:
            return response.text
        else:
            return f"Error: Request failed with status code {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Error occurred: {str(e)}"
    
    

# request_to_scrape Production Code

In [ ]:
# request_to_scrape Production Code
from bs4 import BeautifulSoup

def extract_book_categories(html: str, site: str): 
    soup = BeautifulSoup(html, "html.parser")
    nav_list = extract_element(soup, 'ul', 'nav nav-list')
    print(nav_list.prettify())

---

# Python Execution Code

Develop any code to call the developed functions below.  Add additional cells so you don't need to re-run all of the code when you develop further scripts.

In [48]:
# Python Execution Code
site = "http://books.toscrape.com"
home_page = request_to_scrape(site)

In [59]:
extract_book_categories(home_page, site)

<ul class="nav nav-list">
 <li>
  <a href="catalogue/category/books_1/index.html">
   Books
  </a>
  <ul>
   <li>
    <a href="catalogue/category/books/travel_2/index.html">
     Travel
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/mystery_3/index.html">
     Mystery
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/historical-fiction_4/index.html">
     Historical Fiction
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/sequential-art_5/index.html">
     Sequential Art
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/classics_6/index.html">
     Classics
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/philosophy_7/index.html">
     Philosophy
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/romance_8/index.html">
     Romance
    </a>
   </li>
   <li>
    <a href="catalogue/category/books/womens-fiction_9/index.html">
     Womens Fiction
    </a>
   </li>
   <li>
    <a href="catalogue/categor

## extract_element Production Code

In [58]:
from bs4 import BeautifulSoup

def extract_element(soup, tag: str, class_name=None):
    return soup.find(tag, {"class":class_name})

---

# Jupyter Notebook Test and Linting Set Up

To run `pytest` scripts in a Jupyter Notebook cell, we need to install the `ipytest` package.  This package is NOT required for a pipeline and therefore it can be removed from the `requirements.txt` file before adding the production code to the pipeline.

To run linting, we need to install 2 packages `nbqa` and `flake8`.  We will make sure that `flake8` is included in the `requirements.txt` file when constructing the pipeline so that we can lint as part of the pipeline tests.

Run the following cell to install the `ipytest`, `nbqa` and `flake8` packages and a coverage package to help determine if all of your production code is executed during the tests!

This cell only needs to be run once (or after restarting the notebook kernel) to set up the environment for testing and linting.


In [50]:
# Install the `ipytest`, `nbqa` and `flake8` packages
!pip install ipytest nbqa flake8

### Set up `ipytest` to execute `pytest` scripts in Jupyter Notebook

In [51]:
# Configure ipytest for Jupyter Notebook

import ipytest
ipytest.autoconfig(rewrite_asserts=True, magics=True)

### Create a *config* file for `flake8`

Run this script to create a file in your project root

In [52]:
# Create a config file and ignore some flake8 rules
!echo "[flake8]" > .flake8
!echo "ignore = E402, W291, F811" >> .flake8

# Execute the tests and linting in the Jupyter Notebook

Run the following cell ***EVERY TIME*** you want to run the tests and linting that you have written in the *Python Tests* cell above.

>**Note:**
>
> This entire section does not need to be part of any pipeline scripts.  
> It is only required for the Jupyter Notebook environment during development.


## Run the tests

In [53]:
# Run the tests
ipytest.run("-vv", "-ss")

======================================= test session starts ========================================
platform darwin -- Python 3.13.0, pytest-8.3.3, pluggy-1.5.0 -- /Users/anthonysquire/Digital_Futures/web-scraping-anthony-harry/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/anthonysquire/Digital_Futures/web-scraping-anthony-harry
collecting ... collected 4 items

t_dd5a02443a364c85ac51aa3125b0b6a3.py::test_request_to_scrape PASSED
t_dd5a02443a364c85ac51aa3125b0b6a3.py::test_request_to_scrape_return PASSED
t_dd5a02443a364c85ac51aa3125b0b6a3.py::test_request_to_scrape_fail PASSED
t_dd5a02443a364c85ac51aa3125b0b6a3.py::test_request_to_scrape_get_fail PASSED

======================================== 4 passed in 0.01s =========================================


<ExitCode.OK: 0>

## Run the linter

Run this script each time you want to lint your code

In [54]:
# Run the linter
!nbqa flake8 --show-source --format=pylint webscraping.ipynb

webscraping.ipynb:cell_2:2: [F403] 'from unittest.mock import *' used; unable to detect undefined names
from unittest.mock import *
^
webscraping.ipynb:cell_2:3: [F401] 'unittest' imported but unused
import unittest
^
webscraping.ipynb:cell_2:9: [F405] 'patch' may be undefined, or defined from star imports: unittest.mock
    with patch('requests.get') as mock_get:
         ^
webscraping.ipynb:cell_2:16: [W293] blank line contains whitespace
    
^
webscraping.ipynb:cell_2:18: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:24: [F405] 'patch' may be undefined, or defined from star imports: unittest.mock
    with patch('requests.get') as mock_get:
         ^
webscraping.ipynb:cell_2:34: [W293] blank line contains whitespace
        
^
webscraping.ipynb:cell_2:39: [F405] 'patch' may be undefined, or defined from star imports: unittest.mock
    with patch('requests.get') as mock_get:
         ^
webscraping.ipynb:cell_2:47: [W293] blank line contains whitespace
   


---
